In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

from imblearn.over_sampling import SMOTE

In [4]:
#load Hugo's RF model
rf_pkl = open('./RF_Model_retrained_dec5.pkl', 'rb')
rf_pipeline = pickle.load(rf_pkl);
type(rf_pipeline)
#extract the RF model from the Pipeline object
rf_model = rf_pipeline.steps[1][1];

/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator PolynomialFeatures from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.2 w

In [ ]:
rf_model.predict()

In [10]:
rf_trees = rf_model.estimators_;
depths_to_look = 1;
#count top feature for Random Forest model
rf_top_features = {};
for tree in rf_trees:
    for i in range(depths_to_look):
        top_feat = tree.tree_.feature[i];
        try:
            rf_top_features[top_feat] += 1;
        except:
            rf_top_features[top_feat] = 1;
        
# print("Random Forest Top Features", rf_top_features);
print(len(rf_top_features.keys()), "top features in random forest")

32 top features in random forest


In [11]:
top_feats_sorted = sorted(rf_top_features.items(), key= lambda x: x[1], reverse=True);
print(top_feats_sorted)

[(88, 12), (90, 9), (1, 8), (92, 6), (254, 6), (120, 6), (24, 5), (273, 5), (5, 5), (175, 3), (91, 3), (824, 3), (234, 3), (153, 3), (269, 2), (689, 2), (282, 2), (68, 2), (194, 2), (109, 1), (28, 1), (215, 1), (107, 1), (25, 1), (111, 1), (898, 1), (45, 1), (105, 1), (271, 1), (157, 1), (115, 1), (761, 1)]


In [12]:
#create dataframe that Hugo trained the model on again
df = pd.read_csv("../hugo/cleaned_2014", low_memory = False);
data_train, data_test = train_test_split(df, test_size=.2, stratify=df.paid, random_state=1)
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)

In [13]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False);
poly.fit(x_train_initial);
all_training_features = poly.get_feature_names(input_features= x_train_initial.columns);

In [14]:
print("Training Features: {0}".format(len(all_training_features)))

for i in range(10):
    feat_num = top_feats_sorted[i][0];
    num_split = top_feats_sorted[i][1];
    print("Feature {0}: {1}, {2}".format(feat_num, all_training_features[feat_num], num_split))


Training Features: 989
Feature 88: int_rate dti, 12
Feature 90: int_rate emp_length_1 year, 9
Feature 1: int_rate, 8
Feature 92: int_rate emp_length_2 years, 6
Feature 254: emp_length_1 year emp_length_6 years, 6
Feature 120: int_rate purpose_major_purchase, 6
Feature 24: grade_B, 5
Feature 273: emp_length_1 year purpose_car, 5
Feature 5: emp_length_1 year, 5
Feature 175: dti emp_length_4 years, 3
